### В ноутбуке: строим модели предсказания пола и возраста с spark streaming

[GIT Link](https://github.com/newprolab/sber-spark-ds-10/blob/main/labs/lab04.md)

#### spark.sesion

In [3]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [4]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import *
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("lab04")
         .getOrCreate())

sc = spark.sparkContext

#### Задача
У вас имеются данные: логи посещения пользователей по разным сайтам рунета. По этим пользователям вам также известны: пол и возрастная категория. Вам нужно будет в real-time спрогнозировать эти характеристики по пользователям, о которых у вас нет этой информации, но будут все те же самые логи посещения.

#### Описание данных
Обучающая выборка, с которой вы будете работать, выглядит следующим образом:

```
gender	age	uid	user_json
F	18-24	d50192e5-c44e-4ae8-ae7a-7cfe67c8b777	{"visits": [{"url": "http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun", "timestamp": 1419688144068}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426666298001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426666298000}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426661722001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426661722000}]}
```

- Она расположена на hdfs: `/labs/slaba04/`
- Поле `gender` принимает значения `F` (женщина) и `M` (мужчина)
- Поле `age` принимает значения диапазона возраста: `18-24`, `25-34`, `35-44`, `45-54`, `>=55`
- Поле `uid` принимает значения уникального ID пользователя (cookies): d50192e5-c44e-4ae8-ae7a-7cfe67c8b777.
- Поле `user_json` имеет внутри json со следующей схемой данных: `{"visits": [{"url": "url1", "timestamp": "timestamp1"}, {"url": "url2", "timestamp": "timestamp2"}]}`. В нем содержатся непосредственно логи посещения пользователем страниц вместе с временной меткой посещения.

Данные, по которым вам надо будет построить прогноз (тестовая выборка), хранятся в `Kafka` в таком виде:

```
{"uid": "bdd48781-8243-493d-8ae6-794050a4417f",
 "visits": "[{"url": "http://vk.com/feed", "timestamp": 1425077901001}, {"url": "http://big-cards.a5ltd.com/app/html/vk.html?api_url=http://api.vk.com/api.php&api_id=1804162&api_settings=663823&viewer_id=40849488&viewer_type=2&sid=1b77e9ca975573fed6e31746ca58ea2509f3588d918e7dab2a291f7f7579ae7dab6c21058c837b74b1fbb&secret=37339dfcbf&access_token=06c53bd8f9f9a0afad23b4e166e448495396259fc5a1c5e5243474df82591bd5d0afe877d993ea4a82a81&user_id=40849488&group_id=0&is_app_user=1&auth_key=d7c91ad4bb27e20cb3ed81810f5649db&language=0&parent_language=0&ad_info=elsdcqvcsvftaqxtawjsxht b0q8htjxuvbbjrvbnwojfji2ha8h&is_secure=0&ads_app_id=1804162_903aa103fa48b1e378&referrer=menu&lc_name=2c073eed&hash=", "timestamp": 1425077901000}]"}
```

Название топика, в который мы будем присылать данные: `input_ivan.ivanov`, где вместо `ivan.ivanov` ваш логин от личного кабинета. Данные вам будут поступать по нажатию кнопки "Проверить" в личном кабинете. **Вам топик создавать не нужно**. Он будет автоматом создаваться при записи. Чистить его тоже не надо, потому что Spark Streaming хранит оффсеты, то есть знает, что он уже считывал, что еще не считывал.



Порт брокера Kafka: `6667`. Hostname: `spark-master-1.newprolab.com` или `spark-node-1.newprolab.com`

#### Результат
Вам в свою очередь нужно будет, считывая данные из Kafka, делать прогноз по ним в Spark Streaming и отправлять в real-time эти прогнозные значения в свой другой топик в Kafka в формате:
```
{"uid": "fe1dba8f-3131-439f-9031-851c0da0f126", "gender": "M", "age": "25-34"}
{"uid": "d50192e5-c44e-4ae8-ae7a-7cfe67c8b777", "gender": "F", "age": "18-24"}
```

#### Проверка
Проверка лабы устроена следующим образом:

- Вы готовите свою модель
- Настраиваете чтение данных из одного топика и запись в другой топик
- Запускаете свой скрипт в стриминговым режиме
- Идет подключение к топику на чтение данных
- На странице лабы нажимаете кнопку "Проверить"
- Вам в топик текут данные
- Модель обрабатывает входные данные
- Ваш скрипт сохраняет данные в другой топик
- Чекер на странице лабы выдает результат проверки <br/>
Проверка осуществляется автоматическим скриптом на странице лабы в личном кабинете. Наш чекер в конечном итоге подключится к вашему финальному топику в Kafka и сравнит ваш прогноз с эталонным ответом. На выходе вы получите свой score. В качестве него метрика accuracy: доля пользователей от общего числа, по которым вы верно угадали и пол, и возрастную категорию. Если accuracy будет больше 0.25, то лаба будет засчитана.

In [5]:
# Библиотеки 
import pyspark.sql.types as t
import pyspark.sql.functions as f
from pyspark.ml.feature import HashingTF, Tokenizer, StringIndexer, IndexToString
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

#### get data

In [6]:
# папка с датасетом на HDFS
!hdfs dfs -ls /labs/slaba04/

Found 1 items
-rw-r--r--   3 hdfs hdfs  655090069 2022-01-06 18:46 /labs/slaba04/gender_age_dataset.txt


In [7]:
# датасет для обучения модели
# Note: проверить таргет на наличие пустых значений
!hdfs dfs -cat /labs/slaba04/gender_age_dataset.txt | head -n2

gender	age	uid	user_json
F	18-24	d50192e5-c44e-4ae8-ae7a-7cfe67c8b777	{"visits": [{"url": "http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun", "timestamp": 1419688144068}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426666298001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426666298000}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426661722001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426661722000}]}
cat: Unable to write to output stream.


In [8]:
# формируем spark DF
path = '/labs/slaba04/gender_age_dataset.txt'

schema = t.StructType(fields=[
    t.StructField('gender', t.StringType()),
    t.StructField('age', t.StringType()),
    t.StructField('uid', t.StringType()),
    t.StructField('user_json', t.StringType()),
])

train_data = spark.read.csv(path, header=True, schema=schema, sep='\t')

In [9]:
train_data.show(2)

+------+-----+--------------------+--------------------+
|gender|  age|                 uid|           user_json|
+------+-----+--------------------+--------------------+
|     F|18-24|d50192e5-c44e-4ae...|{"visits": [{"url...|
|     M|25-34|d502331d-621e-472...|{"visits": [{"url...|
+------+-----+--------------------+--------------------+
only showing top 2 rows



In [10]:
# схема для json с визитами
visits_schema = t.StructType([
    t.StructField('visits', t.ArrayType(
        t.StructType([
            t.StructField('url', t.StringType(), True),
            t.StructField('timestamp', t.LongType(), True)
        ])
    ))
])

In [11]:
visits_schema

StructType(List(StructField(visits,ArrayType(StructType(List(StructField(url,StringType,true),StructField(timestamp,LongType,true))),true),true)))

#### feature engineering

In [12]:
# извлечем из визитов пользователя URL, чтобы применить HashingTF
# используем explode, parse_url, collect_list
# parse JSON column + extract URL from visits 

In [13]:
train_data.show(5)

+------+-----+--------------------+--------------------+
|gender|  age|                 uid|           user_json|
+------+-----+--------------------+--------------------+
|     F|18-24|d50192e5-c44e-4ae...|{"visits": [{"url...|
|     M|25-34|d502331d-621e-472...|{"visits": [{"url...|
|     F|25-34|d50237ea-747e-48a...|{"visits": [{"url...|
|     F|25-34|d502f29f-d57a-46b...|{"visits": [{"url...|
|     M| >=55|d503c3b2-a0c2-4f4...|{"visits": [{"url...|
+------+-----+--------------------+--------------------+
only showing top 5 rows



In [14]:
spark_df_flattened = train_data.withColumn("visits", f.from_json(f.col("user_json"), visits_schema))

In [15]:
spark_df_flattened.printSchema()

root
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)
 |-- uid: string (nullable = true)
 |-- user_json: string (nullable = true)
 |-- visits: struct (nullable = true)
 |    |-- visits: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- url: string (nullable = true)
 |    |    |    |-- timestamp: long (nullable = true)



In [16]:
spark_df_flattened= spark_df_flattened.withColumn("visit", f.explode("visits.visits").alias("visit"))

In [17]:
spark_df_flattened.printSchema()

root
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)
 |-- uid: string (nullable = true)
 |-- user_json: string (nullable = true)
 |-- visits: struct (nullable = true)
 |    |-- visits: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- url: string (nullable = true)
 |    |    |    |-- timestamp: long (nullable = true)
 |-- visit: struct (nullable = true)
 |    |-- url: string (nullable = true)
 |    |-- timestamp: long (nullable = true)



[parsing link](https://docs.databricks.com/sql/language-manual/functions/parse_url.html)

In [18]:
spark_df_flattened = ( spark_df_flattened 
    .withColumn("site", f.expr("parse_url(visit.url, 'HOST')")) 
    .drop("visits", "visit","user_json")
                     )

In [19]:
spark_df_flattened.show(5)

+------+-----+--------------------+--------------+
|gender|  age|                 uid|          site|
+------+-----+--------------------+--------------+
|     F|18-24|d50192e5-c44e-4ae...| zebra-zoya.ru|
|     F|18-24|d50192e5-c44e-4ae...|news.yandex.ru|
|     F|18-24|d50192e5-c44e-4ae...|www.sotovik.ru|
|     F|18-24|d50192e5-c44e-4ae...|news.yandex.ru|
|     F|18-24|d50192e5-c44e-4ae...|www.sotovik.ru|
+------+-----+--------------------+--------------+
only showing top 5 rows



In [20]:
spark_df_flattened.show(1, vertical=True, truncate=False)

-RECORD 0--------------------------------------
 gender | F                                    
 age    | 18-24                                
 uid    | d50192e5-c44e-4ae8-ae7a-7cfe67c8b777 
 site   | zebra-zoya.ru                        
only showing top 1 row



In [21]:
spark_df_flattened.count()

5829507

In [22]:
train_final = ( spark_df_flattened 
    .groupBy("gender", "age", "uid") 
    .agg(f.collect_list("site") 
    .alias("sites"))
              )

In [23]:
train_final.show(5)

+------+---+--------------------+--------------------+
|gender|age|                 uid|               sites|
+------+---+--------------------+--------------------+
|     -|  -|13292e10-60bf-435...|[dateandtime.info...|
|     -|  -|13f91463-8386-44c...|[go.mail.ru, pass...|
|     -|  -|1dec593b-4dc5-41b...|[www.sport-expres...|
|     -|  -|1ebc678e-8253-424...|[www.enter.ru, ww...|
|     -|  -|1eecf70e-bb1c-4cb...|[www.licey.net, v...|
+------+---+--------------------+--------------------+
only showing top 5 rows



In [27]:
train_final.count()

41138

In [29]:
train_final.select('uid').distinct().count()

41138

In [30]:
train_final = train_final.na.drop()

In [31]:
train_final.select('uid').distinct().count()

41138

#### pipeline

In [ ]:
# import and clean data + Hashing TF + RandomForest 

In [37]:
X_train, X_test = train_final.randomSplit([0.75, 0.25], seed=123)

In [38]:
hashing_TF = HashingTF(
    inputCol='sites', 
    outputCol='features', 
    numFeatures=10000, 
    binary=False)

In [39]:
indexer_age = StringIndexer(
    inputCol='age', 
    outputCol='ageIndex').fit(train_final)
indexer_gender = StringIndexer(
    inputCol='gender', 
    outputCol='genderIndex').fit(train_final)

In [40]:
rf_age = RandomForestClassifier(
    featuresCol = 'features', 
    labelCol = 'ageIndex', 
    predictionCol='age_pred', 
    rawPredictionCol='age_raw_pred',
    probabilityCol = 'age_probab')

rf_gender = RandomForestClassifier(
    featuresCol = 'features', 
    labelCol = 'genderIndex',
    predictionCol='gender_pred', 
    rawPredictionCol='gender_raw_pred',
    probabilityCol = 'gender_prob')

In [41]:
converter_age = IndexToString(
    inputCol='age_pred', 
    outputCol='PredictedAge', 
    labels=indexer_age.labels)

converter_gender = IndexToString(
    inputCol='gender_pred', 
    outputCol='PredictedGender', 
    labels=indexer_gender.labels)

In [42]:
pipeline = Pipeline(
    stages=[hashing_TF, indexer_age, indexer_gender, rf_age, rf_gender, converter_age, converter_gender])

In [44]:
model = pipeline.fit(train_final)
predictions = model.transform(X_test)

In [ ]:
# calculate accuracy for age
evaluator = MulticlassClassificationEvaluator(labelCol="ageIndex", predictionCol="age_pred")
evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})

In [ ]:
# calculate accuracy for age
evaluator = MulticlassClassificationEvaluator(labelCol="genderIndex", predictionCol="gender_pred")
evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})

#### Kafka

In [45]:
KAFKA_BOOTSTRAP_SERVER = 'spark-node-1.newprolab.com:6667'
KAFKA_INPUT_TOPIC = 'input_anton.ermak'
KAFKA_OUTPUT_TOPIC = 'anton.ermak'

#### static

In [46]:
# чтение в статическом режиме

kafka_read_df = (
    spark.read
    .format('kafka')
    .option('kafka.bootstrap.servers', KAFKA_BOOTSTRAP_SERVER)
    .option('subscribe', KAFKA_INPUT_TOPIC)
    .option('startingOffsets', 'earliest')
    .option('failOnDataLoss', 'False')
    .load()
    .cache()
)

In [47]:
kafka_read_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [48]:
# Парсинг бинарного файла из кафки

event_schema = t.StructType([
    t.StructField('uid', t.StringType(), True),
    t.StructField('visits', t.StringType(), True),
])


visit_schema = t.ArrayType(
    t.StructType([
        t.StructField('url', t.StringType(), True),
        t.StructField('timestamp', t.LongType(), True)
    ])
)


clean_df = (
    kafka_read_df
    .select(f.col('value').cast('string').alias('value'))
    .select(f.from_json(f.col('value'), event_schema).alias('event'))
    .select(
        'event.uid', 
        f.from_json(f.col('event.visits'), visit_schema).alias('visits')
    )
)

clean_df.show(3)

+---+------+
|uid|visits|
+---+------+
+---+------+



In [49]:
# извлечение url
# применение модели, сохранение предсказаний в predictions_df  

prep_df = ( clean_df 
    .withColumn("visit", f.explode("visits").alias("visit")) 
    .withColumn("site", f.expr("parse_url(visit.url, 'HOST')")) 
    .drop("visits", "visit") 
    .groupBy("uid") 
    .agg(f.collect_list("site").alias("sites"))
          )

predictions_df = ( model.transform(prep_df) 
    .select("uid", "PredictedGender", "PredictedAge") 
    .withColumnRenamed("PredictedAge","age") 
    .withColumnRenamed("PredictedGender","gender")
                 )

In [50]:
predictions_df.show(5)

+---+------+---+
|uid|gender|age|
+---+------+---+
+---+------+---+



In [51]:
clean_df.select('uid').distinct().count(), predictions_df.select('uid').distinct().count()

(0, 0)

In [52]:
# Оборачивание предсказания обратно в json

kafka_out_df = (
    predictions_df 
    .select(f.to_json(f.struct(*predictions_df.columns)).alias('value')).limit(5)
)

# Запись в выходной топик

(
    kafka_out_df
    .write
    .format('kafka')
    .option('kafka.bootstrap.servers', KAFKA_BOOTSTRAP_SERVER)
    .option('topic', KAFKA_OUTPUT_TOPIC)
    .save()
)

In [53]:
kafka_out_df.show()

+-----+
|value|
+-----+
+-----+



#### streaming format

In [54]:
# чтение стрима

kafka_stream = (
    spark
    .readStream
    .format('kafka')
    .option('kafka.bootstrap.servers', KAFKA_BOOTSTRAP_SERVER)
    .option('subscribe', KAFKA_INPUT_TOPIC)
    .option('startingOffsets', 'earliest')
    .option('failOnDataLoss', 'False')
    .load()
)

In [55]:
kafka_stream.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [56]:
# предсказание и запись

clean_df = (
    kafka_stream
    .select(f.col('value').cast('string').alias('value'))
    .select(f.from_json(f.col('value'), event_schema).alias('event'))
    .select(
        'event.uid', 
        f.from_json(f.col('event.visits'), visit_schema).alias('visits')
    )
)

prep_df = ( clean_df 
    .withColumn("visit", f.explode("visits").alias("visit")) 
    .withColumn("site", f.expr("parse_url(visit.url, 'HOST')")) 
    .drop("visits", "visit") 
    .groupBy("uid") 
    .agg(f.collect_list("site").alias("sites"))
          )

predictions_df = ( model.transform(prep_df) 
    .select("uid", "PredictedGender", "PredictedAge") 
    .withColumnRenamed("PredictedAge","age") 
    .withColumnRenamed("PredictedGender","gender")
                 )

kafka_write_stream = (
    predictions_df
    .select(f.to_json(f.struct(*predictions_df.columns)).alias('value'))
    .writeStream
    .format("kafka")
    .option("checkpointLocation", "checkpoints/checkpoints_lab04")
    .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVER)
    .option("topic", KAFKA_OUTPUT_TOPIC)
    .outputMode("complete")
    .start()
)

In [51]:
streams = SparkSession.builder.getOrCreate().streams.active
len(streams), streams[0].lastProgress["sources"][0]["description"]

(1, 'KafkaV2[Subscribe[input_anton.ermak]]')

In [53]:
kafka_write_stream.isActive

True

In [54]:
kafka_write_stream.stop()

In [ ]:
spark.stop()